In [1]:
import numpy as np
import pandas as pd

In [2]:
def account_activity(ID, last_record):
    
    """
    Funkcja zwracająca obecny stan konta klienta.
    Konto aktywne - 1
    Konto nieaktywne - 0
    """
        
    if last_record == 0:
        active = 1
    else:
        active = 0
        
        
    return active

In [3]:
def stat2int(status):
    
    """
    text
    text
    """
    
    if status is "5":
        value = 6
    elif status is "4":
        value = 5
    elif status is "3":
        value = 4
    elif status is "2":
        value = 3
    elif status is "1":
        value = 2
    elif status is "0":
        value = 1
    else:
        value = 0
    
    
    return value

In [4]:
def delay_calculator(status_list, historical_range, method):
    
    """
    text
    text
    
    """
    
    status_list = status_list[:historical_range][::-1]
    length = len(status_list)

    delay_list = []

    for i in range(length):
        if i == 0:
            delay_list.append(30*status_list[i])
        else:
            if status_list[i] - 1 == status_list[i-1]:
                delay_list.append(30)
            elif status_list[i] == status_list[i-1]:
                delay_list.append(30*status_list[i])   
            else:
                delay_list.append(30*status_list[i]) 

    if method == "mean":
        delay = np.mean(delay_list)       
    elif method == "median":
        delay = np.median(delay_list)     
    elif method == "median-class":
        value = np.median(delay_list)
        if value in delay_list:
            delay = value
        else:
            while value not in delay_list:
                value = value - 15
            delay = value
            
    return delay

#### Wzór zastosowany do `month_weight`
$a_k=\frac{-2}{n(n+1)}k+\frac{2}{n}$,  
gdzie:  
$n$ - liczba miesięcy  
$k$ - $k$-ty miesiąc  
$a_k$ - waga $k$-tego miesiąca

In [5]:
def month_weight(month, number_of_records, mnor, historical_range):
    
    """
    Funkcja przyporządkowująca wagę danemu miesiącu (month) z uwzględnieniem liczby wszystkich miesięcy (number_of_months).
    Wagi maleją w sposób liniowy, a ich suma daje 1.
    """
    
    k = abs(month) + 1
    n = number_of_records
    
    if n < mnor:
        weight = 0
    elif mnor <= n <= historical_range:
        weight = -2*k/(n*(n+1))+2/n
    else:
        n = historical_range
        if k <= historical_range:
            weight = -2*k/(n*(n+1))+2/n
        else:
            weight = 0
              
                
    return weight

#### Wzór zastosowany w `status2number`  
$a_n=\big(\frac{1}{2}\big)^n-1$  
gdzie:  
$n$ - status z danego miesiąca  
$a_n$ - wartość dla $n$-tego statusu

In [6]:
def status2number(month, status, number_of_records, mnor, historical_range):
    
    """
    W oparciu o funkcję month_weight, status2number przekształca status z wybranego miesiąca na liczbę z przedziału <-1,1>.
    
    """
    
    if month <= historical_range-1:
        weight = month_weight(month,
                              number_of_records,
                              mnor,
                              historical_range)

        if status is "5":
            value = -1
        elif status is "X":
            value = 0
        elif status is "C":
            value = 1
        else:
            n = int(status)
            value = 1/2**(n+1)-1
    else:
        weight = 1
        value = 0
    
    
    return weight * value

In [7]:
def identify_good(status, threshold):
    
    """
    Funkcja określająca czy dany klient może otrzymać kredyt.
    """
    
    if status >= threshold:
        judgment = 1
    else:
        judgment = 0
    
    
    return judgment

In [8]:
def identify_sufficient(status, threshold, second_threshold):

    """
    
    """
    
    if second_threshold <= status < threshold:
        judgment = 1
    else:
        judgment = 0
    
    
    return judgment

In [9]:
def main(data_frame,
         minimum_number_of_records = 4,
         historical_range = 24,
         threshold = 0,
         second_threshold = 0,
         method = "mean"):
    
    """
    text
    text
    
    """
    
    data_frame["MONTHS_BALANCE"] = data_frame["MONTHS_BALANCE"].abs()
    
    extra_frame = data_frame.groupby(by="ID")["MONTHS_BALANCE"].min().reset_index(name="LAST_RECORD")
    data_frame = pd.merge(data_frame, extra_frame, on="ID", how="left")
    
    extra_frame = data_frame.groupby(by="ID")["MONTHS_BALANCE"].count().reset_index(name="NUMBER_OF_RECORDS")
    data_frame = pd.merge(data_frame, extra_frame, on="ID", how="left")
    
    data_frame["MONTHS"] = data_frame["MONTHS_BALANCE"] - data_frame["LAST_RECORD"]
    

    data_frame["ACTIVITY"] = data_frame.apply(lambda df: account_activity(df["ID"],
                                                                        df["LAST_RECORD"]), axis=1)
    
    data_frame["INTEGER_STATUS"] = data_frame.apply(lambda df: stat2int(df["STATUS"]), axis=1)

    status_frame = data_frame.groupby(by="ID")["INTEGER_STATUS"].apply(list).reset_index(name="STATUS_LIST")
    data_frame = pd.merge(data_frame, status_frame, on="ID", how="left")

    data_frame["DELAY_PAYMENT"] = data_frame.apply(lambda df: delay_calculator(df["STATUS_LIST"],
                                                                               historical_range,
                                                                               method), axis=1)
    
    data_frame["PARTIAL_STATUS"] = data_frame.apply(lambda df: status2number(df["MONTHS"],
                                                                             df["STATUS"],
                                                                             df["NUMBER_OF_RECORDS"],
                                                                             minimum_number_of_records,
                                                                             historical_range), axis=1)
    
    extra_frame = data_frame.groupby(by = "ID")["PARTIAL_STATUS"].sum().reset_index(name="TOTAL_STATUS")
    data_frame = pd.merge(data_frame, extra_frame, on="ID", how="left")
    
    data_frame["GOOD"] = data_frame.apply(lambda df: identify_good(df["TOTAL_STATUS"],
                                                                   threshold), axis=1)
    
    if second_threshold != threshold:
        data_frame["SUFFICIENT"] = data_frame.apply(lambda df: identify_sufficient(df["TOTAL_STATUS"],
                                                                                   threshold,
                                                                                   second_threshold), axis=1)

    average_frame = round(data_frame.groupby(by="GOOD")["DELAY_PAYMENT"].mean(), 2).reset_index(name="AVERAGE_DELAY")
    data_frame = pd.merge(data_frame,
                          average_frame,
                          on="GOOD",
                          how="left")
    
    std_frame = round(data_frame.groupby(by="GOOD")["DELAY_PAYMENT"].std(), 2).reset_index(name="STD_DELAY")
    data_frame = pd.merge(data_frame,
                          std_frame,
                          on="GOOD",
                          how="left")
    
    median_frame = round(data_frame.groupby(by="GOOD")["DELAY_PAYMENT"].median(), 2).reset_index(name="MEDIAN_DELAY")
    data_frame = pd.merge(data_frame,
                          median_frame,
                          on="GOOD",
                          how="left")
    
    
    return data_frame

In [10]:
credit_record = pd.read_csv("credit_record.csv")

data = main(credit_record, threshold = 0, second_threshold = -0.5)
data = data[["ID", "ACTIVITY", "GOOD", "SUFFICIENT"]].drop_duplicates(subset="ID")

In [13]:
data.head(10)

,ID,ACTIVITY,GOOD,SUFFICIENT
0,5001711,1,0,1
4,5001712,1,1,0
23,5001713,1,1,0
45,5001714,1,1,0
60,5001715,1,1,0
120,5001717,1,1,0
142,5001718,1,0,1
181,5001719,1,1,0
224,5001720,1,0,0
260,5001723,1,0,1


In [11]:
#data.to_csv("gsb.csv")